In [11]:
import numpy as np
import time

In [12]:
class Node():
    def __init__(self,state,parent,action,depth,step_cost,path_cost,heuristic_cost):
        self.state = state 
        self.parent = parent # parent node
        self.action = action # move up, left, down, right
        self.possible_next_actions = []
        self.depth = depth # depth of the node in the tree
        self.step_cost = step_cost # g(n), the cost to take the step
        self.path_cost = path_cost # accumulated g(n), the cost to reach the current node
        self.heuristic_cost = heuristic_cost # h(n), cost to reach goal state from the current node
        self.get_possible_actions()
        
        # children node
        self.child = None
    
    # see if moving down is valid
    def get_possible_actions(self):
        #self.possible_next_actions = []
        zero_index = tuple(self.state.reshape(1,9)[0]).index(0)
        if zero_index == 0:
            self.possible_next_actions.extend(('R','D'))
        elif zero_index == 1:
            self.possible_next_actions.extend(('R','L','D'))
        elif zero_index ==2:
            self.possible_next_actions.extend(('L','D'))
        elif zero_index == 3:
            self.possible_next_actions.extend(('R','U','D'))
        elif zero_index == 4:
            self.possible_next_actions.extend(('R','L','U','D'))
        elif zero_index == 5:
            self.possible_next_actions.extend(('L','U','D'))
        elif zero_index == 6:
            self.possible_next_actions.extend(('R','U'))
        elif zero_index == 7:
            self.possible_next_actions.extend(('R','L','U'))
        elif zero_index == 8:
            self.possible_next_actions.extend(('L','U'))
            
    def move_piece(self,possible_actions):
        next_node =  list(tuple(self.state.reshape(1,9)[0]))
        index_zero = next_node.index(0)
        if possible_actions == 'U':
            new_index = index_zero - 3
        elif possible_actions == 'D':
            new_index = index_zero + 3
        elif possible_actions == 'L':
            new_index = index_zero - 1
        elif possible_actions == 'R':
            new_index = index_zero + 1
        move_cost = tuple(self.state.reshape(1,9)[0])[new_index]
        next_node[index_zero] = move_cost
        next_node[new_index] = 0
        return np.array(next_node).reshape(3,3), move_cost
        
   
    # return user specified heuristic cost
    def get_h_cost(self,s,g,heuristic_function):
        if heuristic_function == 'h1':
            return self.h1_misplaced_cost(s,g)
        elif heuristic_function == 'h2':
            return self.h2_man_cost(s,g)
        
    # return heuristic cost: number of misplaced tiles
    def h1_misplaced_cost(self,s,g):
        h1 = np.sum(s != g)-1 # minus 1 to exclude the empty tile
        if h1 <= 0:
            return 0
        else:
            return h1
    
    def h2_man_cost(self,state, goal_state):
        heuristic_cost_man = 0
        n = list(tuple(state.reshape(1,9)[0]))
        g = list(tuple(goal_state.reshape(1,9)[0]))
        for x in g:
            heuristic_cost_man += abs(g.index(x) - n.index(x))
        return heuristic_cost_man

        
    # once the goal node is found, trace back to the root node and print out the path
    def print_path(self):
        # create FILO stacks to place the trace
        state_trace = [self.state]
        action_trace = [self.action]
        depth_trace = [self.depth]
        step_cost_trace = [self.step_cost]
        path_cost_trace = [self.path_cost]
        heuristic_cost_trace = [self.heuristic_cost]
        
        # add node information as tracing back up the tree
        while self.parent:
            self = self.parent

            state_trace.append(self.state)
            action_trace.append(self.action)
            depth_trace.append(self.depth)
            step_cost_trace.append(self.step_cost)
            path_cost_trace.append(self.path_cost)
            heuristic_cost_trace.append(self.heuristic_cost)

        # print out the path
        step_counter = 0
        while state_trace:
            print('Step:',step_counter)
            print(state_trace.pop())
            print ('Action=',action_trace.pop(),', Depth=',str(depth_trace.pop()),\
            ', Move Cost=',str(step_cost_trace.pop()),', Total Cost =',\
            str(path_cost_trace.pop() + heuristic_cost_trace.pop()),'\n')
            
            step_counter += 1
                    
    def breadth_first_search(self, goal_state):
        start = time.time()
        
        queue = [self] # queue of found but unvisited nodes, FIFO
        queue_num_nodes_popped = 0 # number of nodes popped off the queue, measuring time performance
        queue_max_length = 1 # max number of nodes in the queue, measuring space performance
        
        depth_queue = [0] # queue of node depth
        path_cost_queue = [0] # queue for path cost
        visited = set([]) # record visited states
        
        while queue:
            # update maximum length of the queue
            if len(queue) > queue_max_length:
                queue_max_length = len(queue)
                
            current_node = queue.pop(0) # select and remove the first node in the queue
            queue_num_nodes_popped += 1 
            
            current_depth = depth_queue.pop(0) # select and remove the depth for current node
            current_path_cost = path_cost_queue.pop(0) # # select and remove the path cost for reaching current node
            visited.add(tuple(current_node.state.reshape(1,9)[0])) # avoid repeated state, which is represented as a tuple
            
            # when the goal state is found, trace back to the root node and print out the path
            if np.array_equal(current_node.state,goal_state):
                current_node.print_path()
                
                print('Time performance:',str(queue_num_nodes_popped),'nodes popped off the queue.')
                print('Space performance:', str(queue_max_length),'nodes in the queue at its max.')
                print('Time spent: %0.2fs' % (time.time()-start))
                return True
            
            else:
                
                for move in current_node.possible_next_actions:
                    new_state, move_value = current_node.move_piece(move)
                    if tuple(new_state.reshape(1,9)[0]) not in visited:
                        current_node.child = Node(state=new_state,parent=current_node,action=move,depth=current_depth+1,\
                                              step_cost=move_value,path_cost=current_path_cost+move_value,heuristic_cost=0)
                        queue.append(current_node.child)
                        depth_queue.append(current_depth+1)
                        path_cost_queue.append(current_path_cost+move_value)
                
    def depth_first_search(self, goal_state):
        start = time.time()
        
        queue = [self] # queue of found but unvisited nodes, FILO
        queue_num_nodes_popped = 0 # number of nodes popped off the queue, measuring time performance
        queue_max_length = 1 # max number of nodes in the queue, measuring space performance
        
        depth_queue = [0] # queue of node depth
        path_cost_queue = [0] # queue for path cost
        visited = set([]) # record visited states
        
        while queue:
            # update maximum length of the queue
            if len(queue) > queue_max_length:
                queue_max_length = len(queue)
                
            current_node = queue.pop(0) # select and remove the first node in the queue
            queue_num_nodes_popped += 1 
            
            current_depth = depth_queue.pop(0) # select and remove the depth for current node
            current_path_cost = path_cost_queue.pop(0) # # select and remove the path cost for reaching current node
            visited.add(tuple(current_node.state.reshape(1,9)[0])) # add state, which is represented as a tuple
            
            # when the goal state is found, trace back to the root node and print out the path
            if np.array_equal(current_node.state,goal_state):
                current_node.print_path()
                
                print('Time performance:',str(queue_num_nodes_popped),'nodes popped off the queue.')
                print('Space performance:', str(queue_max_length),'nodes in the queue at its max.')
                print('Time spent: %0.2fs' % (time.time()-start))
                return True
            
            else:    
                for move in current_node.possible_next_actions:
                    new_state, move_value = current_node.move_piece(move)
                    if tuple(new_state.reshape(1,9)[0]) not in visited:
                        current_node.child = Node(state=new_state,parent=current_node,action=move,depth=current_depth+1,\
                                              step_cost=move_value,path_cost=current_path_cost+move_value,heuristic_cost=0)
                        queue.insert(0,current_node.child)
                        depth_queue.insert(0,current_depth+1)
                        path_cost_queue.insert(0,current_path_cost+move_value)
        
    def iterative_deepening_DFS(self, goal_state):
        start = time.time()
        
        queue_num_nodes_popped = 0 # number of nodes popped off the queue, measuring time performance
        queue_max_length = 1 # max number of nodes in the queue, measuring space performance
        
        # search the tree that's 40 levels in depth
        for depth_limit in range(40):
            #print 'depth limit',depth_limit
        
            queue = [self] # queue of found but unvisited nodes, FILO
            depth_queue = [0] # queue of node depth
            path_cost_queue = [0] # queue for path cost
            visited = set([]) # record visited states

            while queue:
                # update maximum length of the queue
                if len(queue) > queue_max_length:
                    queue_max_length = len(queue)

                current_node = queue.pop(0) # select and remove the first node in the queue
                #print 'pop'
                #print current_node.state
                queue_num_nodes_popped += 1 

                current_depth = depth_queue.pop(0) # select and remove the depth for current node
                #print 'depth:',current_depth,'\n'
                current_path_cost = path_cost_queue.pop(0) # # select and remove the path cost for reaching current node
                visited.add(tuple(current_node.state.reshape(1,9)[0])) # add state, which is represented as a tuple

                # when the goal state is found, trace back to the root node and print out the path
                if np.array_equal(current_node.state,goal_state):
                    current_node.print_path()

                    print('Time performance:',str(queue_num_nodes_popped),'nodes popped off the queue.')
                    print('Space performance:', str(queue_max_length),'nodes in the queue at its max.')
                    print('Time spent: %0.2fs' % (time.time()-start))
                    return True

                else:              
                    if current_depth < depth_limit:
                        
                        for move in current_node.possible_next_actions:
                            new_state, move_value = current_node.move_piece(move)
                            if tuple(new_state.reshape(1,9)[0]) not in visited:
                                current_node.child = Node(state=new_state,parent=current_node,action=move,depth=current_depth+1,\
                                step_cost=move_value,path_cost=current_path_cost+move_value,heuristic_cost=0)
                                queue.insert(0,current_node.child)
                                depth_queue.insert(0,current_depth+1)
                                path_cost_queue.insert(0,current_path_cost+move_value)
                                
    # An extension of BFS that's guided by a prioritized queue based on path cost
    def uniform_cost_search(self, goal_state):
        start = time.time()
        
        queue = [(self,0)] # queue of (found but unvisited nodes, path cost), ordered by path cost(accumulated step cost)
        queue_num_nodes_popped = 0 # number of nodes popped off the queue, measuring time performance
        queue_max_length = 1 # max number of nodes in the queue, measuring space performance
        
        depth_queue = [(0,0)] # queue of node depth, (depth, path cost)
        path_cost_queue = [0] # queue for path cost
        visited = set([]) # record visited states
        
        while queue:
            # sort queue based on path cost, in ascending order
            queue = sorted(queue, key=lambda x: x[1])
            depth_queue = sorted(depth_queue, key=lambda x: x[1])
            path_cost_queue = sorted(path_cost_queue, key=lambda x: x)
            
            # update maximum length of the queue
            if len(queue) > queue_max_length:
                queue_max_length = len(queue)
                
            current_node = queue.pop(0)[0] # select and remove the first node in the queue
            #print 'pop'
            #print current_node.state,'\n'
            queue_num_nodes_popped += 1 
            current_depth = depth_queue.pop(0)[0] # select and remove the depth for current node
            current_path_cost = path_cost_queue.pop(0) # # select and remove the path cost for reaching current node
            visited.add(tuple(current_node.state.reshape(1,9)[0])) # avoid repeated state, which is represented as a tuple
            
            # when the goal state is found, trace back to the root node and print out the path
            if np.array_equal(current_node.state,goal_state):
                current_node.print_path()
                
                print('Time performance:',str(queue_num_nodes_popped),'nodes popped off the queue.')
                print('Space performance:', str(queue_max_length),'nodes in the queue at its max.')
                print('Time spent: %0.2fs' % (time.time()-start))
                return True
            
            else:     
                # see if moving upper tile down is a valid move
                for move in current_node.possible_next_actions:
                    new_state, move_value = current_node.move_piece(move)
                    if tuple(new_state.reshape(1,9)[0]) not in visited:
                        current_node.child = Node(state=new_state,parent=current_node,action=move,depth=current_depth+1,\
                                              step_cost=move_value,path_cost=current_path_cost+move_value,heuristic_cost=0)
                        queue.append((current_node.child,current_path_cost+move_value))
                        depth_queue.append((current_depth+1,current_path_cost+move_value))
                        path_cost_queue.append(current_path_cost+move_value)
    # search based on heuristic cost
    def best_first_search(self, goal_state):
        start = time.time()
        
        queue = [(self,0)] # queue of (found but unvisited nodes, heuristic cost), ordered by heuristic cost
        queue_num_nodes_popped = 0 # number of nodes popped off the queue, measuring time performance
        queue_max_length = 1 # max number of nodes in the queue, measuring space performance
        
        depth_queue = [(0,0)] # queue of node depth, (depth, heuristic cost)
        path_cost_queue = [(0,0)] # queue for path cost, (path_cost, heuristic cost)
        visited = set([]) # record visited states
        
        while queue:
            # sort queue based on heuristic cost, in ascending order
            queue = sorted(queue, key=lambda x: x[1])
            depth_queue = sorted(depth_queue, key=lambda x: x[1])
            path_cost_queue = sorted(path_cost_queue, key=lambda x: x[1])
            
            # update maximum length of the queue
            if len(queue) > queue_max_length:
                queue_max_length = len(queue)
                
            current_node = queue.pop(0)[0] # select and remove the first node in the queue
            #print 'pop'
            #print current_node.state
            #print 'heuristic_cost',current_node.heuristic_cost,'\n'
            
            queue_num_nodes_popped += 1 
            current_depth = depth_queue.pop(0)[0] # select and remove the depth for current node
            current_path_cost = path_cost_queue.pop(0)[0] # # select and remove the path cost for reaching current node
            visited.add(tuple(current_node.state.reshape(1,9)[0])) # avoid repeated state, which is represented as a tuple
            
            # when the goal state is found, trace back to the root node and print out the path
            if np.array_equal(current_node.state,goal_state):
                current_node.print_path()
                
                print('Time performance:',str(queue_num_nodes_popped),'nodes popped off the queue.')
                print('Space performance:', str(queue_max_length),'nodes in the queue at its max.')
                print('Time spent: %0.2fs' % (time.time()-start))
                return True
            
            else:     
                # see if moving upper tile down is a valid move
                for move in current_node.possible_next_actions:
                    new_state, move_value = current_node.move_piece(move)
                    if tuple(new_state.reshape(1,9)[0]) not in visited:
                        h_cost = self.h1_misplaced_cost(new_state,goal_state)
                        current_node.child = Node(state=new_state,parent=current_node,action=move,depth=current_depth+1,\
                                              step_cost=move_value,path_cost=current_path_cost+move_value,heuristic_cost=h_cost)
                        queue.append((current_node.child,h_cost))
                        depth_queue.append((current_depth+1,h_cost))
                        path_cost_queue.append((current_path_cost+move_value,h_cost))
                
     
    # search based on path cost + heuristic cost
    def a_star_search(self,goal_state,heuristic_function):
        start = time.time()
        
        queue = [(self,0)] # queue of (found but unvisited nodes, path cost+heuristic cost), ordered by the second element
        queue_num_nodes_popped = 0 # number of nodes popped off the queue, measuring time performance
        queue_max_length = 1 # max number of nodes in the queue, measuring space performance
        
        depth_queue = [(0,0)] # queue of node depth, (depth, path_cost+heuristic cost)
        path_cost_queue = [(0,0)] # queue for path cost, (path_cost, path_cost+heuristic cost)
        visited = set([]) # record visited states
        
        while queue:
            # sort queue based on path_cost+heuristic cost, in ascending order
            queue = sorted(queue, key=lambda x: x[1])
            depth_queue = sorted(depth_queue, key=lambda x: x[1])
            path_cost_queue = sorted(path_cost_queue, key=lambda x: x[1])
            
            # update maximum length of the queue
            if len(queue) > queue_max_length:
                queue_max_length = len(queue)
                
            current_node = queue.pop(0)[0] # select and remove the first node in the queue
            #print 'pop'
            #print current_node.state
            #print 'path_cost',current_node.path_cost
            #print 'heuristic_cost',current_node.heuristic_cost
            #print 'total_cost',current_node.path_cost+current_node.heuristic_cost,'\n'
            
            queue_num_nodes_popped += 1 
            current_depth = depth_queue.pop(0)[0] # select and remove the depth for current node
            current_path_cost = path_cost_queue.pop(0)[0] # # select and remove the path cost for reaching current node
            visited.add(tuple(current_node.state.reshape(1,9)[0])) # avoid repeated state, which is represented as a tuple
            
            # when the goal state is found, trace back to the root node and print out the path
            if np.array_equal(current_node.state,goal_state):
                current_node.print_path()
                
                print('Time performance:',str(queue_num_nodes_popped),'nodes popped off the queue.')
                print('Space performance:', str(queue_max_length),'nodes in the queue at its max.')
                print('Time spent: %0.2fs' % (time.time()-start))
                return True
            
            else:     
                # see if moving upper tile down is a valid move
                for move in current_node.possible_next_actions:
                    new_state, move_value = current_node.move_piece(move)
                    if tuple(new_state.reshape(1,9)[0]) not in visited:
                        path_cost = current_path_cost + move_value
                        depth = current_depth+1
                        h_cost = self.get_h_cost(new_state,goal_state,heuristic_function)
                        total_cost = path_cost+h_cost
                        current_node.child = Node(state=new_state,parent=current_node,action=move,depth=depth,\
                                              step_cost=move_value,path_cost=path_cost,heuristic_cost=h_cost)
                        queue.append((current_node.child,total_cost))
                        depth_queue.append((depth,total_cost))
                        path_cost_queue.append((path_cost,total_cost))

In [13]:
test = np.array([1,2,3,8,6,4,7,5,0]).reshape(3,3)
easy = np.array([1,3,4,8,6,2,7,0,5]).reshape(3,3)
medium = np.array([2,8,1,0,4,3,7,6,5]).reshape(3,3)
hard = np.array([5,6,7,4,0,8,3,2,1]).reshape(3,3)

initial_state = easy
goal_state = np.array([1,2,3,8,0,4,7,6,5]).reshape(3,3)
print(initial_state,'\n')
print(goal_state)

[[1 3 4]
 [8 6 2]
 [7 0 5]] 

[[1 2 3]
 [8 0 4]
 [7 6 5]]


In [14]:
root_node = Node(state=initial_state,parent=None,action=None,depth=0,step_cost=0,path_cost=0,heuristic_cost=0)

In [15]:
root_node.breadth_first_search(goal_state)

Step: 0
[[1 3 4]
 [8 6 2]
 [7 0 5]]
Action= None , Depth= 0 , Move Cost= 0 , Total Cost = 0 

Step: 1
[[1 3 4]
 [8 0 2]
 [7 6 5]]
Action= U , Depth= 1 , Move Cost= 6 , Total Cost = 6 

Step: 2
[[1 3 4]
 [8 2 0]
 [7 6 5]]
Action= R , Depth= 2 , Move Cost= 2 , Total Cost = 8 

Step: 3
[[1 3 0]
 [8 2 4]
 [7 6 5]]
Action= U , Depth= 3 , Move Cost= 4 , Total Cost = 12 

Step: 4
[[1 0 3]
 [8 2 4]
 [7 6 5]]
Action= L , Depth= 4 , Move Cost= 3 , Total Cost = 15 

Step: 5
[[1 2 3]
 [8 0 4]
 [7 6 5]]
Action= D , Depth= 5 , Move Cost= 2 , Total Cost = 17 

Time performance: 51 nodes popped off the queue.
Space performance: 32 nodes in the queue at its max.
Time spent: 0.01s


True

In [248]:
root_node.uniform_cost_search(easy)

Step: 0
[[1 2 3]
 [8 6 4]
 [7 5 0]]
Action= None , Depth= 0 , Move Cost= 0 , Total Cost = 0 

Step: 1
[[1 2 3]
 [8 6 4]
 [7 0 5]]
Action= L , Depth= 1 , Move Cost= 5 , Total Cost = 5 

Step: 2
[[1 2 3]
 [8 0 4]
 [7 6 5]]
Action= U , Depth= 2 , Move Cost= 6 , Total Cost = 11 

Step: 3
[[1 0 3]
 [8 2 4]
 [7 6 5]]
Action= U , Depth= 3 , Move Cost= 2 , Total Cost = 13 

Step: 4
[[1 3 0]
 [8 2 4]
 [7 6 5]]
Action= R , Depth= 4 , Move Cost= 3 , Total Cost = 16 

Step: 5
[[1 3 4]
 [8 2 0]
 [7 6 5]]
Action= D , Depth= 5 , Move Cost= 4 , Total Cost = 20 

Step: 6
[[1 3 4]
 [8 0 2]
 [7 6 5]]
Action= L , Depth= 6 , Move Cost= 2 , Total Cost = 22 

Step: 7
[[1 3 4]
 [8 6 2]
 [7 0 5]]
Action= D , Depth= 7 , Move Cost= 6 , Total Cost = 28 

Time performance: 80 nodes popped off the queue.
Space performance: 54 nodes in the queue at its max.
Time spent: 0.01s


True

In [249]:
root_node.iterative_deepening_DFS(easy)

Step: 0
[[1 2 3]
 [8 6 4]
 [7 5 0]]
Action= None , Depth= 0 , Move Cost= 0 , Total Cost = 0 

Step: 1
[[1 2 3]
 [8 6 4]
 [7 0 5]]
Action= L , Depth= 1 , Move Cost= 5 , Total Cost = 5 

Step: 2
[[1 2 3]
 [8 0 4]
 [7 6 5]]
Action= U , Depth= 2 , Move Cost= 6 , Total Cost = 11 

Step: 3
[[1 0 3]
 [8 2 4]
 [7 6 5]]
Action= U , Depth= 3 , Move Cost= 2 , Total Cost = 13 

Step: 4
[[1 3 0]
 [8 2 4]
 [7 6 5]]
Action= R , Depth= 4 , Move Cost= 3 , Total Cost = 16 

Step: 5
[[1 3 4]
 [8 2 0]
 [7 6 5]]
Action= D , Depth= 5 , Move Cost= 4 , Total Cost = 20 

Step: 6
[[1 3 4]
 [8 0 2]
 [7 6 5]]
Action= L , Depth= 6 , Move Cost= 2 , Total Cost = 22 

Step: 7
[[1 3 4]
 [8 6 2]
 [7 0 5]]
Action= D , Depth= 7 , Move Cost= 6 , Total Cost = 28 

Time performance: 294 nodes popped off the queue.
Space performance: 7 nodes in the queue at its max.
Time spent: 0.02s


True

In [250]:
root_node.best_first_search(medium)

Step: 0
[[1 2 3]
 [8 6 4]
 [7 5 0]]
Action= None , Depth= 0 , Move Cost= 0 , Total Cost = 0 

Step: 1
[[1 2 3]
 [8 6 4]
 [7 0 5]]
Action= L , Depth= 1 , Move Cost= 5 , Total Cost = 11 

Step: 2
[[1 2 3]
 [8 0 4]
 [7 6 5]]
Action= U , Depth= 2 , Move Cost= 6 , Total Cost = 16 

Step: 3
[[1 2 3]
 [8 4 0]
 [7 6 5]]
Action= R , Depth= 3 , Move Cost= 4 , Total Cost = 19 

Step: 4
[[1 2 0]
 [8 4 3]
 [7 6 5]]
Action= U , Depth= 4 , Move Cost= 3 , Total Cost = 21 

Step: 5
[[1 0 2]
 [8 4 3]
 [7 6 5]]
Action= L , Depth= 5 , Move Cost= 2 , Total Cost = 23 

Step: 6
[[0 1 2]
 [8 4 3]
 [7 6 5]]
Action= L , Depth= 6 , Move Cost= 1 , Total Cost = 24 

Step: 7
[[8 1 2]
 [0 4 3]
 [7 6 5]]
Action= D , Depth= 7 , Move Cost= 8 , Total Cost = 31 

Step: 8
[[8 1 2]
 [4 0 3]
 [7 6 5]]
Action= R , Depth= 8 , Move Cost= 4 , Total Cost = 37 

Step: 9
[[8 0 2]
 [4 1 3]
 [7 6 5]]
Action= U , Depth= 9 , Move Cost= 1 , Total Cost = 38 

Step: 10
[[8 2 0]
 [4 1 3]
 [7 6 5]]
Action= R , Depth= 10 , Move Cost= 2 , To

True

In [253]:
root_node.a_star_search(medium,heuristic_function = 'h2')

Step: 0
[[1 2 3]
 [8 6 4]
 [7 5 0]]
Action= None , Depth= 0 , Move Cost= 0 , Total Cost = 0 

Step: 1
[[1 2 3]
 [8 6 4]
 [7 0 5]]
Action= L , Depth= 1 , Move Cost= 5 , Total Cost = 21 

Step: 2
[[1 2 3]
 [8 0 4]
 [7 6 5]]
Action= U , Depth= 2 , Move Cost= 6 , Total Cost = 21 

Step: 3
[[1 0 3]
 [8 2 4]
 [7 6 5]]
Action= U , Depth= 3 , Move Cost= 2 , Total Cost = 27 

Step: 4
[[0 1 3]
 [8 2 4]
 [7 6 5]]
Action= L , Depth= 4 , Move Cost= 1 , Total Cost = 28 

Step: 5
[[8 1 3]
 [0 2 4]
 [7 6 5]]
Action= D , Depth= 5 , Move Cost= 8 , Total Cost = 32 

Step: 6
[[8 1 3]
 [2 0 4]
 [7 6 5]]
Action= R , Depth= 6 , Move Cost= 2 , Total Cost = 34 

Step: 7
[[8 1 3]
 [2 4 0]
 [7 6 5]]
Action= R , Depth= 7 , Move Cost= 4 , Total Cost = 38 

Step: 8
[[8 1 0]
 [2 4 3]
 [7 6 5]]
Action= U , Depth= 8 , Move Cost= 3 , Total Cost = 37 

Step: 9
[[8 0 1]
 [2 4 3]
 [7 6 5]]
Action= L , Depth= 9 , Move Cost= 1 , Total Cost = 38 

Step: 10
[[0 8 1]
 [2 4 3]
 [7 6 5]]
Action= L , Depth= 10 , Move Cost= 8 , To

True

In [252]:
te = tuple(test.reshape(1,9)[0])